In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Concatenate
from keras.models import Model
from keras.utils import plot_model
from keras.optimizers import Adam
from keras import regularizers
from keras import backend as K
import time
from contextlib import redirect_stdout

In [ ]:
#%run ./CommonFunctions_OthersGeneral.ipynb  ## This is important for library import, getting the path and runlist

# Normal Training

In [ ]:
# Adapted from Markus's codes:
# For Angles

def cyclic_mean_abs_error(y_true, y_pred):
    return K.mean(K.minimum(K.abs(y_pred - y_true),K.minimum(K.abs(y_pred - y_true + 2*np.pi),
                                                             K.abs(y_pred - y_true - 2*np.pi))), axis=-1) 

In [ ]:
# Fourth # Improved
# Combined opions
# Original

def create_cnn_functional_model(angles,num_layers):
    if angles == 'yes':
        input_layer = Input(shape=(108,192,3))

        ## Downsampling here (Flexible to alter)
        X = Conv2D(64, kernel_size=(7,7), activation='relu', strides = (1,1), padding='same')(input_layer)
        for l in range(0, num_layers):
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,1), padding='same')(X)
            X = Dropout(0.25)(X)
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (1,2), padding='same')(X)
            X = Dropout(0.25)(X)
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,2), padding='same')(X)
            X = Dropout(0.25)(X) 

        X = Flatten()(X)
        X = Dense(128, activation="linear", kernel_regularizer=regularizers.l2(0.01), 
                  activity_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(X)
        #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01), 
        #          activity_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005)
        #          ,bias_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005))(X)
        X = Dropout(0.50)(X)

        #output = Dense(3, activation="linear")(X) # linear is default
        #model = Model(inputs=input_layer, outputs=output)
        output1_xy = Dense(units=2, activation="linear",name='output1_xy')(X) # linear is default
        output2_angles = Dense(units=1, activation="linear",name='output2_angles')(X) # linear is default
        model = Model(inputs=input_layer, outputs=[output1_xy,output2_angles]) 

        dt = time.strftime("%Y-%m-%d_%H.%M.%S")

        # summarize layers
        from contextlib import redirect_stdout
        file = dir14 + '/' + test_runNum + '-' + dt +'.txt'
        with open(file, 'w') as f:
            with redirect_stdout(f):
                model.summary()
        print(model.summary())


        # plot graph
        plot_model(model, to_file=(dir11 + test_runNum + '-convolutional_neural_network_' + dt + '.png'))

        model.compile(optimizer=Adam(lr=0.00001),loss={'output1_xy':'mae','output2_angles':cyclic_mean_abs_error},
                      metrics=['mse','acc'])


        return model
    
    elif angles == 'no':
        input_layer = Input(shape=(108,192,3))
    
        ## Downsampling here (Flexible to alter)
        X = Conv2D(64, kernel_size=(7,7), activation='relu', strides = (1,1), padding='same')(input_layer)
        for l in range(0, num_layers):
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,1), padding='same')(X)
            X = Dropout(0.25)(X)
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (1,2), padding='same')(X)
            X = Dropout(0.25)(X)
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,2), padding='same')(X)
            X = Dropout(0.25)(X) 

        X = Flatten()(X)
        #X = Dense(128, activation="linear")(X) #X = Dense(128)(X) # linear is default
        X = Dense(dense, activation="linear")(X)
        # OR
        #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001), 
        #          activity_regularizer=regularizers.l2(0.001),bias_regularizer=regularizers.l2(0.001))(X)
        # OR
        #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), 
        #          activity_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
        #          ,bias_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(X)
        #X = Dropout(0.50)(X)

        output = Dense(2, activation="linear")(X) # linear is default
        model = Model(inputs=input_layer, outputs=output)

        dt = time.strftime("%Y-%m-%d_%H.%M.%S")

        # summarize layers
        from contextlib import redirect_stdout
        #file = dir14 + '/' + dt +'.txt'
        file = dir14 + '/' + test_runNum + '-' + dt +'.txt'
        with open(file, 'w') as f:
            with redirect_stdout(f):
                model.summary()
        print(model.summary())


        # plot graph
        plot_model(model, to_file=(dir11 + test_runNum + '-convolutional_neural_network_' + dt + '.png'))

        model.compile(optimizer=Adam(lr=0.00001),loss='mae', metrics=['mse','acc'])


        return model


# Model Verification

# TEMP for replot of failed model #

from keras import backend as K

def mish(x):
    return x * K.tanh(K.softplus(x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'mish': Activation(mish)})

# Mainly have modified the part without including angles for my initial report
# Please modify according to need if necessary
# Did not include this part in my report at last because cannot think of a valid reason in explaining the result

def create_cnn_functional_model_WeightsPerturb(angles,num_layers,randomise,randomiseMethod):
    if angles == 'yes':
        input_layer = Input(shape=(108,192,3))

        ## Downsampling here (Flexible to alter)
        X = Conv2D(64, kernel_size=(7,7), activation='relu', strides = (1,1), padding='same')(input_layer)
        for l in range(0, num_layers):
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,1), padding='same')(X)
            X = Dropout(0.25)(X)
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (1,2), padding='same')(X)
            X = Dropout(0.25)(X)
            X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,2), padding='same')(X)
            X = Dropout(0.25)(X) 

        X = Flatten()(X)
        X = Dense(128, activation="linear", kernel_regularizer=regularizers.l2(0.01), 
                  activity_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(X)
        #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01), 
        #          activity_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005)
        #          ,bias_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005))(X)
        X = Dropout(0.50)(X)

        #output = Dense(3, activation="linear")(X) # linear is default
        #model = Model(inputs=input_layer, outputs=output)
        output1_xy = Dense(units=2, activation="linear",name='output1_xy')(X) # linear is default
        output2_angles = Dense(units=1, activation="linear",name='output2_angles')(X) # linear is default
        model = Model(inputs=input_layer, outputs=[output1_xy,output2_angles]) 

        dt = time.strftime("%Y-%m-%d_%H.%M.%S")

        # summarize layers
        from contextlib import redirect_stdout
        file = dir14 + '/' + test_runNum + '-' + dt +'.txt'
        with open(file, 'w') as f:
            with redirect_stdout(f):
                model.summary()
        print(model.summary())


        # plot graph
        plot_model(model, to_file=(dir11 + test_runNum + '-convolutional_neural_network_' + dt + '.png'))

        model.compile(optimizer=Adam(lr=0.00001),loss={'output1_xy':'mae','output2_angles':cyclic_mean_abs_error},
                      metrics=['mse','acc'])


        return model
    
    elif angles == 'no':
        if randomise == 'no':
            input_layer = Input(shape=(108,192,3))
    
            ## Downsampling here (Flexible to alter)
            X = Conv2D(64, kernel_size=(7,7), activation='relu', strides = (1,1), padding='same')(input_layer)
            for l in range(0, num_layers):
                X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,1), padding='same')(X)
                X = Dropout(0.25)(X)
                X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (1,2), padding='same')(X)
                X = Dropout(0.25)(X)
                X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,2), padding='same')(X)
                X = Dropout(0.25)(X) 

            X = Flatten()(X)
            #X = Dense(128, activation="linear")(X) #X = Dense(128)(X) # linear is default
            X = Dense(dense, activation="linear")(X)
            # OR
            #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001), 
            #          activity_regularizer=regularizers.l2(0.001),bias_regularizer=regularizers.l2(0.001))(X)
            # OR
            #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), 
            #          activity_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
            #          ,bias_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(X)
            #X = Dropout(0.50)(X)

            output = Dense(2, activation="linear")(X) # linear is default
            model = Model(inputs=input_layer, outputs=output)

            dt = time.strftime("%Y-%m-%d_%H.%M.%S")

            # summarize layers
            from contextlib import redirect_stdout
            #file = dir14 + '/' + dt +'.txt'
            file = dir14 + '/' + test_runNum + '-' + dt +'.txt'
            with open(file, 'w') as f:
                with redirect_stdout(f):
                    model.summary()
            print(model.summary())


            # plot graph
            plot_model(model, to_file=(dir11 + test_runNum + '-convolutional_neural_network_' + dt + '.png'))

            #model.compile(optimizer=Adam(lr=0.00001),loss='mae', metrics=['mse','acc'])
            model.compile(optimizer=Adam(lr=lr_rate),loss='mae', metrics=['mse','acc'])
        
        elif randomise == 'yes': ## Added to try the randomization ##
            if randomiseMethod == '1': # just permutation, not helpful to get what we want
                input_layer = Input(shape=(108,192,3))
    
                ## Downsampling here (Flexible to alter)
                X = Conv2D(64, kernel_size=(7,7), activation='relu', strides = (1,1), padding='same')(input_layer)
                for l in range(0, num_layers):
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,1), padding='same')(X)
                    X = Dropout(0.25)(X)
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (1,2), padding='same')(X)
                    X = Dropout(0.25)(X)
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,2), padding='same')(X)
                    X = Dropout(0.25)(X) 

                X = Flatten()(X)
                #X = Dense(128, activation="linear")(X) #X = Dense(128)(X) # linear is default
                X = Dense(dense, activation="linear")(X)
                # OR
                #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001), 
                #          activity_regularizer=regularizers.l2(0.001),bias_regularizer=regularizers.l2(0.001))(X)
                # OR
                #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), 
                #          activity_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
                #          ,bias_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(X)
                #X = Dropout(0.50)(X)

                output = Dense(2, activation="linear")(X) # linear is default
                model = Model(inputs=input_layer, outputs=output)

                dt = time.strftime("%Y-%m-%d_%H.%M.%S")

                # summarize layers
                from contextlib import redirect_stdout
                #file = dir14 + '/' + dt +'.txt'
                file = dir14 + '/' + test_runNum + '-' + dt +'.txt'
                with open(file, 'w') as f:
                    with redirect_stdout(f):
                        model.summary()
                print(model.summary())


                # plot graph
                #plot_model(model, to_file=(dir11 + 'convolutional_neural_network_' + dt + '.png'))
                plot_model(model, to_file=(dir11 + test_runNum + '-convolutional_neural_network_' + dt + '.png'))

                model.compile(optimizer=Adam(lr=0.00001),loss='mae', metrics=['mse','acc'])
        
        
                # 1. Personalized randomisation: https://gist.github.com/jkleint/eb6dc49c861a1c21b612b568dd188668
                # https://github.com/keras-team/keras/issues/341

                initial_weights = model.get_weights()
                print('Initial weights:\n', initial_weights)

                def shuffle_weights(model, weights=None):

                    #"""Randomly permute the weights in `model`, or the given `weights`.
                    #This is a fast approximation of re-initializing the weights of a model.
                    #Assumes weights are distributed independently of the dimensions of the weight tensors
                    #  (i.e., the weights have the same distribution along each dimension).
                    #:param Model model: Modify the weights of the given model.
                    #:param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
                    #  If `None`, permute the model's current weights.
                    #"""
                    if weights is None:
                        weights = model.get_weights()
                    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
                    # Faster, but less random: only permutes along the first dimension
                    # weights = [np.random.permutation(w) for w in weights]
                    model.set_weights(weights)

                for rnd in range(25):
                    shuffle_weights(model, initial_weights)
                    print('\nRound {} starting weights:\n'.format(rnd), model.get_weights())
            
            elif randomiseMethod == '2': # perturbing directly over the inputs with kernel and bias initializers
                input_layer = Input(shape=(108,192,3))
    
                ## Downsampling here (Flexible to alter)
                X = Conv2D(64, kernel_size=(7,7), activation='relu', strides = (1,1), padding='same')(input_layer)
                for l in range(0, num_layers):
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,1), padding='same',
                               kernel_initializer='he_normal', bias_initializer='ones')(X)
                    X = Dropout(0.25)(X)
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (1,2), padding='same',
                              kernel_initializer='he_normal', bias_initializer='ones')(X)
                    X = Dropout(0.25)(X)
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,2), padding='same',
                              kernel_initializer='he_normal', bias_initializer='ones')(X)
                    X = Dropout(0.25)(X) 

                X = Flatten()(X)
                #X = Dense(128, activation="linear")(X) #X = Dense(128)(X) # linear is default
                X = Dense(dense, activation="linear")(X)
                # OR
                #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001), 
                #          activity_regularizer=regularizers.l2(0.001),bias_regularizer=regularizers.l2(0.001))(X)
                # OR
                #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), 
                #          activity_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
                #          ,bias_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(X)
                #X = Dropout(0.50)(X)

                output = Dense(2, activation="linear")(X) # linear is default
                model = Model(inputs=input_layer, outputs=output)

                dt = time.strftime("%Y-%m-%d_%H.%M.%S")

                # summarize layers
                from contextlib import redirect_stdout
                #file = dir14 + '/' + dt +'.txt'
                file = dir14 + '/' + test_runNum + '-' + dt +'.txt'
                with open(file, 'w') as f:
                    with redirect_stdout(f):
                        model.summary()
                print(model.summary())


                # plot graph
                plot_model(model, to_file=(dir11 + test_runNum + '-convolutional_neural_network_' + dt + '.png'))

                model.compile(optimizer=Adam(lr=0.00001),loss='mae', metrics=['mse','acc'])
                # 2. Use initializer:
                 # https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79
                 # https://machinelearningmastery.com/why-initialize-a-neural-network-with-random-weights/
                 # https://keras.io/initializers/
                 # https://stackoverflow.com/questions/56630174/how-to-re-initialize-layer-weights-of-an-existing-model-in-keras
                 # https://www.tensorflow.org/api_docs/python/tf/keras/initializers/Initializer
                 # https://blog.goodaudience.com/visualizing-various-filter-initializers-in-keras-ca14c996db22
                 # https://keras.rstudio.com/articles/guide_keras.html
        
            elif randomiseMethod == '3': # set initial weights to zeros and set bias to the inputs
                input_layer = Input(shape=(108,192,3))
    
                ## Downsampling here (Flexible to alter)
                X = Conv2D(64, kernel_size=(7,7), activation='relu', strides = (1,1), padding='same')(input_layer)
                for l in range(0, num_layers):
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,1), padding='same',
                               bias_initializer='ones')(X)
                    X = Dropout(0.25)(X)
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (1,2), padding='same',
                               bias_initializer='ones')(X)
                    X = Dropout(0.25)(X)
                    X = Conv2D(64*(l+1), kernel_size=(7,7), activation='relu', strides = (2,2), padding='same',
                               bias_initializer='ones')(X)
                    X = Dropout(0.25)(X) 

                X = Flatten()(X)
                #X = Dense(128, activation="linear")(X) #X = Dense(128)(X) # linear is default
                X = Dense(dense, activation="linear")(X)
                # OR
                #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001), 
                #          activity_regularizer=regularizers.l2(0.001),bias_regularizer=regularizers.l2(0.001))(X)
                # OR
                #X = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), 
                #          activity_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
                #          ,bias_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(X)
                #X = Dropout(0.50)(X)

                output = Dense(2, activation="linear")(X) # linear is default
                model = Model(inputs=input_layer, outputs=output)

                dt = time.strftime("%Y-%m-%d_%H.%M.%S")

                # summarize layers
                from contextlib import redirect_stdout
                #file = dir14 + '/' + dt +'.txt'
                file = dir14 + '/' + test_runNum + '-' + dt +'.txt'
                with open(file, 'w') as f:
                    with redirect_stdout(f):
                        model.summary()
                print(model.summary())


                # plot graph
                plot_model(model, to_file=(dir11 + test_runNum + '-convolutional_neural_network_' + dt + '.png'))

                model.compile(optimizer=Adam(lr=0.00001),loss='mae', metrics=['mse','acc'])
                
                
                #model.layers[i].set_weights(listOfNumpyArrays)    
                #model.get_layer(layerName).set_weights(...)
                #model.set_weights(listOfNumpyArrays)
                
                #https://keras.io/models/about-keras-models/
                
                
                initial_weights = model.get_weights()
                print('Initial weights:\n', initial_weights)

                def set_weightsZeros(model, weights):#=None):
                    
                    #if weights is None:
                    #    weights = model.get_weights()
                    weights = [np.zeros(w.shape) for w in weights]
                    model.set_weights(weights)

                set_weightsZeros(model, initial_weights)
                print('New weights:\n', model.get_weights())
                
                
                
            # 4. Read https://arxiv.org/pdf/1602.05931.pdf


        return model